In [47]:
from datetime import timedelta, datetime
import rsa_headers as rh
import rsa_data as rd
import main
import config
import uuid
from pandasql import sqldf

files = r"C:\Users\MB2705851\Desktop\SYNTELL TEST\0006_20211231.RSA"
import pandas as pd
pd.set_option("display.max_columns", None)

testfile = r"C:\Users\MB2705851\Desktop\Temp\SYNTELL-test.csv"

In [44]:
def to_df(file: str) -> pd.DataFrame:
    df = pd.read_csv(file, header=None, sep="\n")
    df = df[0].str.split("\s+|,\s+|,", expand=True)
    df = pd.DataFrame(df)
    return df

In [3]:
def get_head(df) -> pd.DataFrame:
    dfh = pd.DataFrame(
        df.loc[
            (df[0].isin(["H0", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
            | (
                (df[0].isin(["21", "70", "30", "13", "60"]))
                & (~df[1].isin(["0", "1", "2", "3", "4"]))
            )
            | (
                (df[0].isin(["10"]))
                & (df[1].isin(["1", "8", "5", "01", "08", "05"]))
            )
        ]
    ).dropna(axis=1, how="all")
    dfh["index"] = dfh.index
    breaks = dfh["index"].diff() != 1
    groups = breaks.cumsum()
    dfh["newindex"] = groups
    dfh = dfh.set_index("newindex")
    dfh = dfh.drop(columns=["index"])
    return dfh

def headers(dfh: pd.DataFrame) -> pd.DataFrame:
    if not dfh.empty:
        headers = pd.DataFrame()
        headers["site_id"] = dfh.loc[dfh[0] == "S0", 1].astype(str)
        if not dfh.loc[dfh[0] == "S1", 1:].empty:
            headers["station_name"] = (
                dfh.loc[dfh[0] == "S1", 1:]
                .dropna(axis=1)
                .apply(" ".join, axis=1)
                .astype(str)
            )
        else:
            headers["station_name"] = (
                dfh.loc[dfh[0] == "S0", 2:]
                .dropna(axis=1)
                .apply(" ".join, axis=1)
                .astype(str)
            )

        try:
            headers["y"] = dfh.loc[dfh[0] == "S0", 5].astype(float)
            headers["x"] = dfh.loc[dfh[0] == "S0", 6].astype(float)
        except Exception:
            pass

        headers["number_of_lanes"] = dfh.loc[dfh[0] == "L0", 2].astype(int)

        try:
            headers["speedbin1"] = dfh.loc[dfh[0] == "21", 4].astype(int)
            headers["speedbin2"] = dfh.loc[dfh[0] == "21", 5].astype(int)
            headers["speedbin3"] = dfh.loc[dfh[0] == "21", 6].astype(int)
            headers["speedbin4"] = dfh.loc[dfh[0] == "21", 7].astype(int)
            headers["speedbin5"] = dfh.loc[dfh[0] == "21", 8].astype(int)
            headers["speedbin6"] = dfh.loc[dfh[0] == "21", 9].astype(int)
            headers["speedbin7"] = dfh.loc[dfh[0] == "21", 10].astype(int)
            headers["speedbin8"] = dfh.loc[dfh[0] == "21", 11].astype(int)
            headers["speedbin9"] = dfh.loc[dfh[0] == "21", 12].astype(int)
            headers["type_21_count_interval_minutes"] = dfh.loc[
                dfh[0] == "21", 1
            ].astype(int)
            headers["type_21_programmable_rear_to_rear_headway_bin"] = dfh.loc[
                dfh[0] == "21", 3
            ].astype(int)
            headers["type_21_program_id"] = "2"
        except Exception:
            pass
        try:
            headers["type_10_vehicle_classification_scheme_primary"] = dfh.loc[
                dfh[0] == "10", 1
            ].astype(int)
            headers["type_10_vehicle_classification_scheme_secondary"] = dfh.loc[
                dfh[0] == "10", 2
            ].astype(int)
            headers["type_10_maximum_gap_milliseconds"] = dfh.loc[
                dfh[0] == "10", 3
            ].astype(int)
            headers["type_10_maximum_differential_speed"] = dfh.loc[
                dfh[0] == "10", 4
            ].astype(int)
        except Exception:
            pass
        try:
            headers["type_30_summary_interval_minutes"] = dfh.loc[
                dfh[0] == "30", 2
            ].astype(int)
            headers["type_30_vehicle_classification_scheme"] = dfh.loc[
                dfh[0] == "30", 3
            ].astype(int)
        except Exception:
            pass
        try:
            headers["type_70_summary_interval_minutes"] = dfh.loc[
                dfh[0] == "70", 1
            ].astype(int)
            headers["type_70_vehicle_classification_scheme"] = dfh.loc[
                dfh[0] == "70", 2
            ].astype(int)
            headers["type_70_maximum_gap_milliseconds"] = dfh.loc[
                dfh[0] == "70", 3
            ].astype(int)
            headers["type_70_maximum_differential_speed"] = dfh.loc[
                dfh[0] == "70", 4
            ].astype(int)
            headers["type_70_error_bin_code"] = dfh.loc[dfh[0] == "70", 5].astype(
                int
            )
        except Exception:
            pass

        if not dfh.loc[dfh[0] == "D3", 1].empty:
            headers["start_datetime"] = dfh.loc[dfh[0] == "D3", 1].astype(str)
            headers["start_time"] = dfh.loc[dfh[0] == "D3", 2].astype(str)
            headers["end_datetime"] = dfh.loc[dfh[0] == "D3", 3].astype(str)
            headers["end_time"] = dfh.loc[dfh[0] == "D3", 4].astype(str)
        else:
            headers["start_datetime"] = dfh.loc[dfh[0] == "D1", 1].astype(str)
            headers["start_time"] = dfh.loc[dfh[0] == "D1", 2].astype(str)
            headers["end_datetime"] = dfh.loc[dfh[0] == "D1", 3].astype(str)
            headers["end_time"] = dfh.loc[dfh[0] == "D1", 4].astype(str)

        try:
            for idx, row in headers.iterrows():
                if len(row['start_datetime']) == 6 and len(row['start_time']) == 6:
                    row['start_datetime'] = pd.to_datetime(row['start_datetime'] + row['start_time'], format='%y%m%d%H%M%S')
                elif len(row['start_datetime']) == 8 and len(row['start_time']) == 6:
                    row['start_datetime'] = pd.to_datetime(row['start_datetime'] + row['start_time'], format='%Y%m%d%H%M%S')
                elif len(row['start_datetime']) == 6 and len(row['start_time']) == 8:
                    row['start_datetime'] = pd.to_datetime(row['start_datetime'] + row['start_time'], format='%y%m%d%H%M%S%f')
                elif len(row['start_datetime']) == 8 and len(row['start_time']) == 8:
                    row['start_datetime'] = pd.to_datetime(row['start_datetime'] + row['start_time'], format='%Y%m%d%H%M%S%f')


                if len(row['end_datetime']) == 6 and row['end_time'].strip('0') == '24':
                    row['end_datetime'] = pd.to_datetime(row['end_datetime'], format='%y%m%d')+timedelta(days=1)
                elif len(row['end_datetime']) ==8 and row['end_time'].strip('0') == '24':
                    row['end_datetime'] = pd.to_datetime(row['end_datetime'], format='%Y%m%d')+timedelta(days=1)
                elif len(row['end_datetime']) == 6 and len(row['end_time']) == 6:
                    row['start_datetime'] = pd.to_datetime(row['end_datetime'] + row['end_time'], format='%y%m%d%H%M%S')
                elif len(row['end_datetime']) == 8 and len(row['end_time']) == 6:
                    row['start_datetime'] = pd.to_datetime(row['end_datetime'] + row['end_time'], format='%Y%m%d%H%M%S')
                elif len(row['end_datetime']) == 6 and len(row['end_time']) == 8:
                    row['start_datetime'] = pd.to_datetime(row['end_datetime'] + row['end_time'], format='%y%m%d%H%M%S%f')
                elif len(row['end_datetime']) == 8 and len(row['end_time']) == 8:
                    row['start_datetime'] = pd.to_datetime(row['end_datetime'] + row['end_time'], format='%Y%m%d%H%M%S%f')   
            
        except:
            headers["end_datetime"] = headers.apply(
                lambda x: pd.to_datetime(
                    x["end_datetime"] + x["end_time"], format="%y%m%d%H%M%S"
                )
                if (
                    x["end_time"] != "240000"
                    and len(x["end_datetime"]) == 6
                    and len(x["end_time"]) == 6
                )
                else (
                    pd.to_datetime(
                        x["end_datetime"] + x["end_time"], format="%y%m%d%H%M%S%f"
                    )
                    if (
                        x["end_time"] != "24000000"
                        and len(x["end_datetime"]) == 6
                        and len(x["end_time"]) == 8
                    )
                    else (
                        pd.to_datetime(
                            x["end_datetime"] + x["end_time"], format="%Y%m%d%H%M%S"
                        )
                        if (
                            x["end_time"] != "240000"
                            and len(x["end_datetime"]) == 8
                            and len(x["end_time"]) == 6
                        )
                        else (
                            pd.to_datetime(
                                x["end_datetime"] + x["end_time"],
                                format="%Y%m%d%H%M%S%f",
                            )
                            if (
                                x["end_time"] != "24000000"
                                and len(x["end_datetime"]) == 8
                                and len(x["end_time"]) == 8
                            )
                            else (
                                pd.to_datetime(x["end_datetime"], format="%y%m%d")
                                + timedelta(days=1)
                                if (
                                    x["end_time"] == "240000"
                                    and len(x["end_datetime"]) == 6
                                    and len(x["end_time"]) == 6
                                )
                                else (
                                    pd.to_datetime(x["end_datetime"], format="%y%m%d")
                                    + timedelta(days=1)
                                    if (
                                        x["end_time"] == "24000000"
                                        and len(x["end_datetime"]) == 6
                                        and len(x["end_time"]) == 8
                                    )
                                    else (
                                        pd.to_datetime(
                                            x["end_datetime"], format="%Y%m%d"
                                        )
                                        + timedelta(days=1)
                                        if (
                                            x["end_time"] == "240000"
                                            and len(x["end_datetime"]) == 8
                                            and len(x["end_time"]) == 6
                                        )
                                        else (
                                            pd.to_datetime(
                                                x["end_datetime"], format="%Y%m%d"
                                            )
                                            + timedelta(days=1)
                                            if (
                                                x["end_time"] == "24000000"
                                                and len(x["end_datetime"]) == 8
                                                and len(x["end_time"]) == 8
                                            )
                                            else pd.to_datetime(
                                                x["end_datetime"] + x["end_time"]
                                            )
                                        )
                                    )
                                )
                            )
                        )
                    )
                ),
                axis=1,
            )

            headers["start_datetime"] = headers.apply(
                lambda x: pd.to_datetime(
                    x["start_datetime"] + x["start_time"], format="%y%m%d%H%M%S"
                )
                if (
                    x["start_time"] != "240000"
                    and len(x["start_datetime"]) == 6
                    and len(x["start_time"]) == 6
                )
                else (
                    pd.to_datetime(
                        x["start_datetime"] + x["start_time"], format="%y%m%d%H%M%S%f"
                    )
                    if (
                        x["start_time"] != "24000000"
                        and len(x["start_datetime"]) == 6
                        and len(x["start_time"]) == 8
                    )
                    else (
                        pd.to_datetime(
                            x["start_datetime"] + x["start_time"], format="%Y%m%d%H%M%S"
                        )
                        if (
                            x["start_time"] != "240000"
                            and len(x["start_datetime"]) == 8
                            and len(x["start_time"]) == 6
                        )
                        else (
                            pd.to_datetime(
                                x["start_datetime"] + x["start_time"],
                                format="%Y%m%d%H%M%S%f",
                            )
                            if (
                                x["start_time"] != "24000000"
                                and len(x["start_datetime"]) == 8
                                and len(x["start_time"]) == 8
                            )
                            else (
                                pd.to_datetime(x["start_datetime"], format="%y%m%d")
                                + timedelta(days=1)
                                if (
                                    x["start_time"] == "240000"
                                    and len(x["start_datetime"]) == 6
                                    and len(x["start_time"]) == 6
                                )
                                else (
                                    pd.to_datetime(x["start_datetime"], format="%y%m%d")
                                    + timedelta(days=1)
                                    if (
                                        x["start_time"] == "24000000"
                                        and len(x["start_datetime"]) == 6
                                        and len(x["start_time"]) == 8
                                    )
                                    else (
                                        pd.to_datetime(
                                            x["start_datetime"], format="%Y%m%d"
                                        )
                                        + timedelta(days=1)
                                        if (
                                            x["start_time"] == "240000"
                                            and len(x["start_datetime"]) == 8
                                            and len(x["start_time"]) == 6
                                        )
                                        else (
                                            pd.to_datetime(
                                                x["start_datetime"], format="%Y%m%d"
                                            )
                                            + timedelta(days=1)
                                            if (
                                                x["start_time"] == "24000000"
                                                and len(x["start_datetime"]) == 8
                                                and len(x["start_time"]) == 8
                                            )
                                            else pd.to_datetime(
                                                x["start_datetime"] + x["start_time"]
                                            )
                                        )
                                    )
                                )
                            )
                        )
                    )
                ),
                axis=1,
            )

        
        
        headers = headers.drop(["start_time"], axis=1)
        headers = headers.drop(["end_time"], axis=1)

        headers["start_datetime"] = pd.to_datetime(headers["start_datetime"])
        headers["end_datetime"] = pd.to_datetime(headers["end_datetime"])
        headers["site_id"] = headers["site_id"].astype(str)

        try:
            headers["instrumentation_description"] = (
                dfh.loc[dfh[0] == "I0", 1:]
                .dropna(axis=1)
                .apply(" ".join, axis=1)
                .astype(str)
            )
        except Exception:
            headers["instrumentation_description"] = None

        try:
            headers["type_30_summary_interval_minutes"] = headers[
                "type_21_count_interval_minutes"
            ]
        except Exception:
            pass
        try:
            headers["type_70_summary_interval_minutes"] = headers[
                "type_21_count_interval_minutes"
            ]
            headers["type_70_vehicle_classification_scheme"] = headers[
                "type_21_count_interval_minutes"
            ]
            headers["type_70_vehicle_classification_scheme"] = headers[
                "type_21_count_interval_minutes"
            ]
        except Exception:
            pass

        headers = headers.fillna(method="ffill")
        headers = headers.fillna(method="bfill")

        headers = headers.drop_duplicates(ignore_index=True)

        headers["header_id"] = ""
        headers["header_id"] = headers["header_id"].apply(
            lambda x: str(uuid.uuid4())
        )

    else:
        pass
    return headers


In [4]:
df = to_df(files)

In [5]:
dfh = get_head(df)

In [82]:
header = headers(dfh)

In [36]:
def dtype21(df: pd.DataFrame) -> pd.DataFrame:
    data = df.loc[(df[0] == "21") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
        axis=1, how="all"
    )
    dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
        axis=1, how="all"
    )

    if data.empty:
        pass
    else:
        if (data[1] == "0").any():
            ddf = data.iloc[:, 2:]
            ddf = pd.DataFrame(ddf).dropna(axis=1, how="all")
            ddf.columns = [
                "end_datetime",
                "end_time",
                "duration_min",
                "lane_number",
                "speedbin1",
                "speedbin2",
                "speedbin3",
                "speedbin4",
                "speedbin5",
                "speedbin6",
                "speedbin7",
                "speedbin8",
                "speedbin9",
                "speedbin10",
                "sum_of_heavy_vehicle_speeds",
                "short_heavy_vehicles",
                "medium_heavy_vehicles",
                "long_heavy_vehicles",
                "rear_to_rear_headway_shorter_than_2_seconds",
                "rear_to_rear_headways_shorter_than_programmed_time",
            ]
            ddf["speedbin0"] = 0

        elif (data[1] == "1").any():
            ddf = data.iloc[:, 3:]
            ddf = pd.DataFrame(ddf).dropna(axis=1, how="all")
            ddf.columns = [
                "end_datetime",
                "end_time",
                "duration_min",
                "lane_number",
                "speedbin0",
                "speedbin1",
                "speedbin2",
                "speedbin3",
                "speedbin4",
                "speedbin5",
                "speedbin6",
                "speedbin7",
                "speedbin8",
                "speedbin9",
                "speedbin10",
                "sum_of_heavy_vehicle_speeds",
                "short_heavy_vehicles",
                "medium_heavy_vehicles",
                "long_heavy_vehicles",
                "rear_to_rear_headway_shorter_than_2_seconds",
                "rear_to_rear_headways_shorter_than_programmed_time",
            ]

        ddf = ddf.fillna(0)
        ddf["duration_min"] = ddf["duration_min"].astype(int)
        ddf["lane_number"] = ddf["lane_number"].astype(int)
        ddf["speedbin0"] = ddf["speedbin0"].astype(int)
        ddf["speedbin1"] = ddf["speedbin1"].astype(int)
        ddf["speedbin2"] = ddf["speedbin2"].astype(int)
        ddf["speedbin3"] = ddf["speedbin3"].astype(int)
        ddf["speedbin4"] = ddf["speedbin4"].astype(int)
        ddf["speedbin5"] = ddf["speedbin5"].astype(int)
        ddf["speedbin6"] = ddf["speedbin6"].astype(int)
        ddf["speedbin7"] = ddf["speedbin7"].astype(int)
        ddf["speedbin8"] = ddf["speedbin8"].astype(int)
        ddf["speedbin9"] = ddf["speedbin9"].astype(int)
        ddf["speedbin10"] = ddf["speedbin10"].astype(int)
        ddf["sum_of_heavy_vehicle_speeds"] = ddf[
            "sum_of_heavy_vehicle_speeds"
        ].astype(int)
        ddf["short_heavy_vehicles"] = ddf["short_heavy_vehicles"].astype(int)
        ddf["medium_heavy_vehicles"] = ddf["medium_heavy_vehicles"].astype(int)
        ddf["long_heavy_vehicles"] = ddf["long_heavy_vehicles"].astype(int)
        ddf["rear_to_rear_headway_shorter_than_2_seconds"] = ddf[
            "rear_to_rear_headway_shorter_than_2_seconds"
        ].astype(int)
        ddf["rear_to_rear_headways_shorter_than_programmed_time"] = ddf[
            "rear_to_rear_headways_shorter_than_programmed_time"
        ].astype(int)

        ddf["total_heavy_vehicles_type21"] = (
            ddf["short_heavy_vehicles"]
            + ddf["medium_heavy_vehicles"]
            + ddf["long_heavy_vehicles"]
        )
        ddf["total_heavy_vehicles_type21"] = ddf[
            "total_heavy_vehicles_type21"
        ].astype(int)

        ddf["total_light_vehicles_type21"] = (
            ddf["speedbin1"]
            + ddf["speedbin2"]
            + ddf["speedbin3"]
            + ddf["speedbin4"]
            + ddf["speedbin5"]
            + ddf["speedbin6"]
            + ddf["speedbin7"]
            + ddf["speedbin8"]
            + ddf["speedbin9"]
            + ddf["speedbin10"]
            - ddf["short_heavy_vehicles"]
            - ddf["medium_heavy_vehicles"]
            - ddf["long_heavy_vehicles"]
        )
        ddf["total_light_vehicles_type21"] = ddf[
            "total_light_vehicles_type21"
        ].astype(int)

        ddf["total_vehicles_type21"] = (
            ddf["speedbin1"]
            + ddf["speedbin2"]
            + ddf["speedbin3"]
            + ddf["speedbin4"]
            + ddf["speedbin5"]
            + ddf["speedbin6"]
            + ddf["speedbin7"]
            + ddf["speedbin8"]
            + ddf["speedbin9"]
            + ddf["speedbin10"]
        )
        ddf["total_vehicles_type21"] = ddf["total_vehicles_type21"].astype(int)

        max_lanes = ddf["lane_number"].max()
        ddf["direction"] = ddf.apply(
            lambda x: "P" if x["lane_number"] <= (int(max_lanes) / 2) else "N",
            axis=1,
        )

        direction = dfh2.loc[dfh2[0] == "L1", 1:3].astype(int)
        direction = direction.drop_duplicates()
        try:
            ddf["forward_direction_code"] = ddf.apply(
                lambda x: Data.get_direction(x["lane_number"], direction), axis=1
            )
            # FIXME: ddf['lane_position_code']=ddf.apply(lambda x: Data.get_lane_position(x['lane_number'],direction),axis=1)
        except Exception:
            ddf["forward_direction_code"] = None
            # ddf['lane_position_code']=None

        ddf["end_datetime"] = ddf.apply(
            lambda x: pd.to_datetime(
                x["end_datetime"] + x["end_time"], format="%y%m%d%H%M"
            )
            if (
                x["end_time"] != "2400"
                and len(str(x["end_datetime"])) == 6
                and len(x["end_time"]) == 4
            )
            else (
                pd.to_datetime(
                    x["end_datetime"] + x["end_time"], format="%Y%m%d%H%M"
                )
                if (
                    x["end_time"] != "2400"
                    and len(str(x["end_datetime"])) == 8
                    and len(x["end_time"]) == 4
                )
                else (
                    pd.to_datetime(x["end_datetime"], format="%y%m%d")
                    + timedelta(days=1)
                    if (
                        x["end_time"] == "2400"
                        and len(str(x["end_datetime"])) == 6
                        and len(x["end_time"]) == 4
                    )
                    else (
                        pd.to_datetime(x["end_datetime"], format="%Y%m%d")
                        + timedelta(days=1)
                        if (
                            x["end_time"] != "2400"
                            and len(str(x["end_datetime"])) == 8
                            and len(x["end_time"]) == 4
                        )
                        else pd.to_datetime(x["end_datetime"])
                    )
                )
            ),
            axis=1,
        )
        ddf = ddf.drop(["end_time"], axis=1)
        ddf["start_datetime"] = pd.to_datetime(
            ddf["end_datetime"]
        ) - pd.to_timedelta(ddf["duration_min"].astype(int), unit="m")
        # ddf['year'] = ddf['start_datetime'].dt.year
        t1 = dfh2.loc[dfh2[0] == "S0", 1].unique()
        ddf["site_id"] = str(t1[0])

        ddf = ddf.drop_duplicates()
        ddf["start_datetime"] = ddf["start_datetime"].astype("datetime64[ns]")

        return ddf


In [45]:
def join(header: pd.DataFrame, data: pd.DataFrame) -> pd.DataFrame:
    if data.empty:
        df = pd.DataFrame()
    else:
        q = """
		SELECT header.header_id, header.station_name, data.*
		FROM header
		LEFT JOIN data ON data.start_datetime WHERE data.start_datetime >= header.start_datetime AND data.end_datetime <= header.end_datetime;
		"""
        q2 = """UPDATE data set header_id = (SELECT header_id from header WHERE data.start_datetime >= header.start_datetime AND data.counttime_end <= header.enddate)"""
        pysqldf = lambda q: sqldf(q, globals())
        df = sqldf(q, locals())
        df = pd.DataFrame(df)
    return df


def data_join(data: pd.DataFrame, header: pd.DataFrame) -> pd.DataFrame:
    if data is None:
        pass
    elif data.empty:
        pass
    else:
        data = pd.DataFrame(data)
        data = join(header, data)
    return data


In [83]:
data = data_join(ddf, header)
data.drop("station_name", axis=1, inplace=True)

In [33]:
ddf['total_vehicles_type21'].groupby(ddf['start_datetime'].dt.to_period('D')).sum().mean()

42185.58064516129

In [110]:
def header_calcs(header, data, dtype):
    data['start_datetime'] = pd.to_datetime(data['start_datetime'])
    if dtype == 21:
        header['adt_total'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['header_id']]).sum().mean()
        header['adt_positive_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum().mean()
        header['adt_positive_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()

        header['adtt_total'] = data['total_heavy_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['header_id']]).sum().mean()
        header['adtt_positive_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum().mean()
        header['adtt_positive_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()

        header['total_vehicles'] = data['total_vehicles_type21'].groupby(data['header_id']).sum()[0]
        header['total_vehicles_positive_direction'] = data['total_vehicles_type21'].groupby(data['direction'].loc[data['direction'] == 'P']).sum()[0]
        header['total_vehicles_positive_direction'] = data['total_vehicles_type21'].groupby(data['direction'].loc[data['direction'] == 'N']).sum()[0]

        header['total_heavy_vehicles'] = data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[0]
        header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[0]
        header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum()[0]
        header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[0] / data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[0]
        header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum()[0] / data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[0]

        header['total_light_vehicles'] = data['total_light_vehicles_type21'].groupby(data['header_id']).sum()[0]
        header['total_light_positive_direction'] = data['total_light_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum()[0]
        header['total_light_positive_direction'] = data['total_light_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[0]

        header['short_heavy_vehicles'] = data['short_heavy_vehicles'].groupby(data['header_id']).sum()[0]
        header['short_heavy_positive_direction'] = data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum()[0]
        header['short_heavy_positive_direction'] = data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[0]

        header['Medium_heavy_vehicles'] = data['medium_heavy_vehicles'].groupby(data['header_id']).sum()[0]
        header['Medium_heavy_positive_direction'] = data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[0]
        header['Medium_heavy_positive_direction'] = data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum()[0]

        header['long_heavy_vehicles'] = data['long_heavy_vehicles'].groupby(data['header_id']).sum()[0]
        header['long_heavy_positive_direction'] = data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum()[0]
        header['long_heavy_positive_direction'] = data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[0]

        header['vehicles_with_rear_to_rear_headway_less_than_2sec_positive_dire'] = data['rear_to_rear_headway_shorter_than_2_seconds'].groupby([data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum()[0]
        header['vehicles_with_rear_to_rear_headway_less_than_2sec_negative_dire'] = data['rear_to_rear_headway_shorter_than_2_seconds'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[0]
        header['vehicles_with_rear_to_rear_headway_less_than_2sec_total'] = data['rear_to_rear_headway_shorter_than_2_seconds'].groupby(data['header_id']).sum()[0]
    
        header['type_21_count_interval_minutes'] = data['duration_min'].mean()

        header['highest_volume_per_hour_positive_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum().max()
        header['highest_volume_per_hour_negative_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().max()
        header['highest_volume_per_hour_total'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['header_id']]).sum().max()

        header['15th_highest_volume_per_hour_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum().quantile(q=0.15,  interpolation='linear'))
        header['15th_highest_volume_per_hour_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().quantile(q=0.15,  interpolation='linear'))
        header['15th_highest_volume_per_hour_total'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['header_id']]).sum().quantile(q=0.15, interpolation='linear'))
        
        header['30th_highest_volume_per_hour_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum().quantile(q=0.3,  interpolation='linear'))
        header['30th_highest_volume_per_hour_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().quantile(q=0.3, interpolation='linear'))
        header['30th_highest_volume_per_hour_total'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['header_id']]).sum().quantile(q=0.3, interpolation='linear'))

        # header['average_speed_positive_direction'] = 
        # header['average_speed_negative_direction'] = 
        header['average_speed'] = ((
            (header['speedbin1'] * data['speedbin1'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin2'] * data['speedbin2'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin3'] * data['speedbin3'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin4'] * data['speedbin4'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin5'] * data['speedbin5'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin6'] * data['speedbin6'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin7'] * data['speedbin7'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin8'] * data['speedbin8'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin9'] * data['speedbin9'].groupby(data['header_id']).sum()[0]) 
            )
            / data['sum_of_heavy_vehicle_speeds'].groupby(data['header_id']).sum()[0]
            )
        # header['average_speed_light_vehicles_positive_direction'] = 
        # header['average_speed_light_vehicles_negative_direction'] = 
        header['average_speed_light_vehicles'] = ((
            (header['speedbin1'] * data['speedbin1'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin2'] * data['speedbin2'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin3'] * data['speedbin3'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin4'] * data['speedbin4'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin5'] * data['speedbin5'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin6'] * data['speedbin6'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin7'] * data['speedbin7'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin8'] * data['speedbin8'].groupby(data['header_id']).sum()[0]) +
            (header['speedbin9'] * data['speedbin9'].groupby(data['header_id']).sum()[0]) -
            data['sum_of_heavy_vehicle_speeds'].groupby(data['header_id']).sum()[0]
            )
            / data['sum_of_heavy_vehicle_speeds'].groupby(data['header_id']).sum()[0]
            )
        
        # header['average_speed_heavy_vehicles_positive_direction'] = 
        # header['average_speed_heavy_vehicles_negative_direction'] = 
        # header['average_speed_heavy_vehicles'] = 

        header['truck_split_positive_direction'] = (str(round(data['short_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'P']]).sum()[0] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'P']]).sum()[0])) + ' : ' +
        str(round(data['medium_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'P']]).sum()[0] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'P']]).sum()[0])) + ' : ' +
        str(round(data['long_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'P']]).sum()[0] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'P']]).sum()[0]))
        )
        header['truck_split_negative_direction'] = (str(round(data['short_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[0] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[0])) + ' : ' +
        str(round(data['medium_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[0] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[0])) + ' : ' +
        str(round(data['long_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[0] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[0]))
        )
        header['truck_split_total'] = (str(round(data['short_heavy_vehicles'].groupby(data['header_id']).sum()[0] / 
        data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[0])) + ' : ' +
        str(round(data['medium_heavy_vehicles'].groupby(data['header_id']).sum()[0] / 
        data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[0])) + ' : ' +
        str(round(data['long_heavy_vehicles'].groupby(data['header_id']).sum()[0] / 
        data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[0]))
        )

        return header
    elif dtype == 30:
        if header['adt_total'].isnull():
            header['adt_total'] = data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['header_id']]).sum().mean()
            header['adt_positive_direction'] = data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum().mean()
            header['adt_positive_direction'] = data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()
        else:
            pass

        if header['adtt_total'].isnull():
            header['adtt_total'] = data['total_heavy_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['header_id']]).sum().mean()
            header['adtt_positive_direction'] = data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum().mean()
            header['adtt_positive_direction'] = data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()
        else:
            pass

        if header['total_vehicles'].isnull():
            header['total_vehicles'] = data['total_vehicles_type30'].groupby(data['header_id']).sum()[0]
            header['total_vehicles_positive_direction'] = data['total_vehicles_type30'].groupby(data['direction'].loc[data['direction'] == 'P']).sum()[0]
            header['total_vehicles_positive_direction'] = data['total_vehicles_type30'].groupby(data['direction'].loc[data['direction'] == 'N']).sum()[0]
        else:
            pass
        
        if header['total_heavy_vehicles'].isnull():
            header['total_heavy_vehicles'] = data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[0]
            header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[0]
            header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum()[0]
            header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[0] / data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[0]
            header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum()[0] / data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[0]
        else:
            pass

        if header['total_light_vehicles'].isnull():
            header['total_light_vehicles'] = data['total_light_vehicles_type30'].groupby(data['header_id']).sum()[0]
            header['total_light_positive_direction'] = data['total_light_vehicles_type30'].groupby([data['direction'].loc[data['direction'] == 'P'], data['header_id']]).sum()[0]
            header['total_light_positive_direction'] = data['total_light_vehicles_type30'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[0]
        else:
            pass

        return header

    elif dtype == 70:
    
        return header

    elif dtype == 10:
    
        return header

    elif dtype == 60:
        
        return header

    else:
        return header

In [111]:
header_out = header_calcs(header, data, 21)

In [122]:
header_out['type_10_vehicle_classification_scheme_primary'].isnull()

0    True
Name: type_10_vehicle_classification_scheme_primary, dtype: bool